In [1]:
!pip install pytorch_lightning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 727.0/727.0 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 763.4/763.4 kB 46.0 MB/s eta 0:00:00


In [21]:
!pwd

/content


In [23]:
!git branch -m master

In [24]:
!git remote add origin https://github.com/chumthea97/pytorch-lightning.git

In [25]:
!git add .

In [33]:
!git commit -m "first commit"

[master (root-commit) a5219db] first commit
 60 files changed, 51055 insertions(+)
 create mode 100644 .config/.last_opt_in_prompt.yaml
 create mode 100644 .config/.last_survey_prompt.yaml
 create mode 100644 .config/.last_update_check.json
 create mode 100644 .config/active_config
 create mode 100644 .config/config_sentinel
 create mode 100644 .config/configurations/config_default
 create mode 100644 .config/default_configs.db
 create mode 100644 .config/gce
 create mode 100644 .config/logs/2023.08.31/13.24.06.550663.log
 create mode 100644 .config/logs/2023.08.31/13.24.32.047854.log
 create mode 100644 .config/logs/2023.08.31/13.24.55.974256.log
 create mode 100644 .config/logs/2023.08.31/13.25.03.338259.log
 create mode 100644 .config/logs/2023.08.31/13.25.27.753261.log
 create mode 100644 .config/logs/2023.08.31/13.25.28.491021.log
 create mode 100644 dataset/MNIST/raw/t10k-images-idx3-ubyte
 create mode 100644 dataset/MNIST/raw/t10k-images-idx3-ubyte.gz
 create mode 100644 dataset

In [2]:
!pip install torchmetrics

In [20]:
import torch
import pytorch_lightning as pl
import torch.nn.functional as F
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch import nn, optim
from torch.utils.data import DataLoader
from tqdm import tqdm
from torch.utils.data import random_split
import torchmetrics
from torchmetrics import Metric


class NN(nn.Module):
    def __init__(self, input_size, num_classes):
        super().__init__()
        self.fc1 = nn.Linear(input_size, 50)
        self.fc2 = nn.Linear(50, num_classes)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

class MyAccuracy(Metric):
  def __init__(self):
    super().__init__()
    self.add_state("total", default=torch.tensor(0), dist_reduce_fx="sum")
    self.add_state("correct", default=torch.tensor(0), dist_reduce_fx="sum")

  def update(self, preds, targets):
    preds = torch.argmax(preds, dim=1)
    assert preds.shape == targets.shape
    self.correct += sum(preds == targets)
    self.total += targets.numel()

  def compute(self):
    return self.correct.float() / self.total.float()

class NN(pl.LightningModule):
  def __init__(self, input_size, num_classes):
    super().__init__()
    self.fc1 = nn.Linear(input_size, 50)
    self.fc2 = nn.Linear(50, num_classes)
    self.loss_fn = nn.CrossEntropyLoss()
    self.accuracy = torchmetrics.Accuracy(task="multiclass", num_classes=num_classes)
    self.my_accuracy = MyAccuracy()
    self.f1_score = torchmetrics.F1Score(task="multiclass", num_classes=num_classes)

  def forward(self, x):
    x = F.relu(self.fc1(x))
    x = self.fc2(x)
    return x

  # activate trainer
  def training_step(self, batch, batch_idx):
    loss, score, y = self._common_step(batch, batch_idx)
    self.log("Training Loss: ", loss)
    accuracy = self.my_accuracy(score, y)#self.accuracy(score, y)
    f1_score = self.f1_score(score, y)
    self.log_dict({'train_loss': loss, 'accuracy': accuracy, 'train_f1_score': f1_score}, on_step=False, on_epoch=True, prog_bar=True, logger=True)
    return {"loss":loss, "score":score, "y":y}

  # take all output and compute some metrices, average loss for each batch
  def on_training_epoch_end(self, output):
    scores = torch.cat([x["scores"] for x in outputs])
    y = torch.cat([x["y"] for x in outputs])
    self.log_dict(
            {
                "train_acc": self.accuracy(scores, y),
                "train_f1": self.f1_score(scores, y),
            },
            on_step=False,
            on_epoch=True,
            prog_bar=True,
        )



  def validation_step(self, batch, batch_idx):
    loss, score, y = self._common_step(batch, batch_idx)
    self.log("Validate Loss: ", loss)
    return loss

  def test_step(self, batch, batch_idx):
    loss, score, y = self._common_step(batch, batch_idx)
    self.log("Test Loss: ", loss)
    return loss

  def _common_step(self, batch, batch_idx):
    x, y = batch
    x = x.reshape(x.shape[0], -1) # 784 MNIST Image
    score = self.forward(x)
    loss = self.loss_fn(score, y)
    return loss, score, y

  def prediction_step(self, batch, batch_idx):
    x, y = batch
    x = x.reshape(x.shpae[0], -1) # 784 MNIST Image
    score = self.forward(x)
    preds = torch.argmax(score, dim=1)
    self.log("Predictions : ", preds)
    return preds


  def configure_optimizers(self):
    return optim.Adam(self.parameters(), lr=0.0001)

# class CustomDataset():
#   def __init__():

#   def __getitem__():

#   def __len__():


class MNISTModule(pl.LightningModule):
  def __init__(self, data_dir, batch_size, num_worker):
    super().__init__()
    self.data_dir   = data_dir
    self.batch_size = batch_size
    self.num_worker = num_worker


  # downlaod data, custom data ...
  def prepare_data(self):
    # SIngle GPU
    datasets.MNIST(self.data_dir, train=True, download=True, transform= transforms.ToTensor())
    datasets.MNIST(self.data_dir, train=False, download=True, transform= transforms.ToTensor())

  def setup(self, stage):
    # multiple GPU
    entire_dataset = datasets.MNIST(
        root=self.data_dir,
        train=True,
        transform=transforms.ToTensor(),
        download=False
    )

    self.train_ds, self.val_ds = random_split(entire_dataset, [50000, 10000])

    self.test_ds = datasets.MNIST(
        root=self.data_dir,
        train=False,
        transform=transforms.ToTensor(),
        download=False
    )

  def train_dataloader(self):
    return DataLoader(
        self.train_ds,
        batch_size=self.batch_size,
        num_workers=self.num_worker,
        shuffle=True
    )

  def val_dataloader(self):
    return DataLoader(
        self.val_ds,
        batch_size=self.batch_size,
        num_workers=self.num_worker,
        shuffle=False
    )

  def test_dataloader(self):
    return DataLoader(
        self.test_ds,
        batch_size=self.batch_size,
        num_workers=self.num_worker,
        shuffle=False
    )


# Set device cuda for GPU if it's available otherwise run on the CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hyperparameters
input_size = 784
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epochs = 3

# Initialize network
model = NN(input_size=input_size, num_classes=num_classes).to(device=device)

dm = MNISTModule(data_dir='dataset/', batch_size=batch_size, num_worker=3)

trainer = pl.Trainer(accelerator="gpu", devices=[0], min_epochs=1, max_epochs=2, precision=16)
trainer.fit(model, datamodule=dm)
trainer.validate(model, dm)
trainer.test(model, dm)

# # Check accuracy on training & test to see how good our model
# model.to(device)
# print(f"Accuracy on training set: {check_accuracy(train_loader, model)*100:.2f}")
# print(f"Accuracy on validation set: {check_accuracy(val_loader, model)*100:.2f}")
# print(f"Accuracy on test set: {check_accuracy(test_loader, model)*100:.2f}")

/usr/local/lib/python3.10/dist-packages/lightning_fabric/connector.py:554: UserWarning: 16 is supported for historical reasons but its usage is discouraged. Please set your precision to 16-mixed instead!
  rank_zero_warn(
INFO:pytorch_lightning.utilities.rank_zero:Using 16bit Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name        | Type               | Params
---------------------------------------------------
0 | fc1         | Linear             | 39.2 K
1 | fc2         | Linear             | 510   
2 | loss_fn     | CrossEntropyLoss   | 0     


Sanity Checking: 0it [00:00, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 3 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Validate metric      ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│      Validate Loss:       │    0.44679248332977295    │
└───────────────────────────┴───────────────────────────┘

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│        Test Loss:         │    0.4135950803756714     │
└───────────────────────────┴───────────────────────────┘

[{'Test Loss: ': 0.4135950803756714}]